In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy.linalg as la
import scipy.sparse as sparse
from matplotlib import cm
import timeit
import numba
from numba import jit

In [2]:
#output settings
np.set_printoptions(precision=2)
%matplotlib qt 

In [3]:
def u_analytical(x, y):

    uan = np.sin(np.pi*x)*np.sin(np.pi*y)

    return uan

def source(x, y):

    f = -2*np.pi**2*np.sin(np.pi*x)*np.sin(np.pi*y)

    return f

In [4]:
def sor_iteration(matrix, source, grid, init_guess=None, tolerance=1.e-8, itermax=1000, omega=1.5):

    M, f, u0, rtol, kmax, w = matrix, source, init_guess, tolerance, itermax, omega

    x, y = grid

    Nx, Ny = x.shape[1], y.shape[0]

    dx, dy = (x[0, -1] - x[0, 0])/Nx, (y[-1, 0] - y[0, 0])/Ny

    if u0 is None:

        u0 = np.zeros_like(f)
    
    u = u0
    k = 0
    rel_diff = tol + 1
    conv_hist = []
    while rel_diff > rtol and k < kmax:

        u_prev = u.copy()
        for j in range(1, Ny-1):
            for i in range(1, Nx-1):
        #         # if i != j:
        #         u[j, i] = ((u[j-1, i] + u[j+1, i])*dx**2 + (u[j, i-1] + u[j, i+1])*dy**2 - dx**2*dy**2*f[j,i])/(2*(dx**2 +                                                                                                                               dy**2))
                u[j, i] = (1-omega)*u[j,i] + omega*0.25*(u[j-1, i] + u[j+1, i] + u[j, i-1] + u[j, i+1] - dx**2*f[j, i])

        # for j in range(1,Ny-1):
        #     for i in range(1, Nx-1):
        #         R = (u[j,i-1]+u[j-1,i]+u[j,i+1]+u[j+1,i]-4.0*u[j,i] - dx**2*f[j, i])
        #         du = 0.25*R
        #         u[j,i]+=du

        rel_diff = la.norm(u-u_prev)/la.norm(u_prev)
        conv_hist.append(rel_diff)
        k += 1

    return u, k, rel_diff, conv_hist

In [5]:
Nx = 40
Ny = 40

x_I, x_F = 0, 1
y_I, y_F = 0, 1

X = np.linspace(x_I, x_F, Nx+1)
Y = np.linspace(y_I, y_F, Ny+1)

x, y = np.meshgrid(X, Y)

In [6]:
#Test the solver
f = source(x, y)                          #source term
u_guess = np.cos(np.pi*x)*np.cos(np.pi*y)             #initial guess
itermax = 2000000                            #maximum number of iterations
tol = 1e-8                               #desired tolerance
omega_opt = 2/(1+np.sin(np.pi/Nx))
print(omega_opt)

start = timeit.default_timer()            #time the solver

u, iternum, rel_diff, conv_hist = sor_iteration(np.eye(Nx), f, (x, y), init_guess=None, 
                                                itermax=itermax, tolerance=tol, omega=1)       #calculate the solution

end = timeit.default_timer()

elapsed = (end - start)

u_an = u_analytical(x, y)                #analytical solution for comparison

if iternum == itermax:

    print('WARNING: desired tolerance has not been reached for the given maximum iterations \n')

print('Number of iterations: {} \n'
      'L2 norm between last two iterations: {:1.2e} \n'
      'Time elapsed: {:1.2e} s'.format(iternum, rel_diff, elapsed))

1.8544977810681016
Number of iterations: 2161 
L2 norm between last two iterations: 9.96e-09 
Time elapsed: 9.07e+00 s


In [7]:
u_an = u_analytical(x, y)

error = la.norm(u - u_an, 2)/la.norm(u_an, 2)

print(error)

0.047697710465012144


In [8]:
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')


scat = ax.scatter(x, y, u, c = 'r', label = 'numerical', alpha = 1)
surf = ax.plot_surface(x, y, u_an, cmap = cm.coolwarm, label = 'analytical', alpha = 0.3)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('$u = \sin\pi x \sin\pi y$')
# ax.set_zlim(top=2., bottom=-0.5)

surf._facecolors2d=surf._facecolors3d
surf._edgecolors2d=surf._edgecolors3d
ax.legend(loc = 2)
plt.show()